In [ ]:
# Nicole's code starts here 
# Importing "The Numbers" data & cleaning it up

In [1]:
import pandas as pd
import numpy as np
from config import OMB_api_key
import requests
import json
from pprint import pprint

In [14]:
numbers_df = pd.read_csv('DataFiles/TheNumbers_Original.csv')
print(numbers_df.shape)
numbers_df.head(1)

(2000, 19)


,Domestic Release Date,Worldwide Release Date,Year Released (Domestic),Year Released (Worldwide),Title,Theatrical Distributor,Genre,Source,Production Method,Creative Type,Production Budget,Opening Weekend Theaters,Maximum Theaters,Theatrical Engagements,Opening Weekend Revenue,Domestic Box Office,Infl. Adj. Dom. Box Office,International Box Office,Worldwide Box Office
0,18-Jun-10,18-Jun-10,2010,2010,Toy Story 3,Walt Disney,Adventure,Original Screenplay,Digital Animation,Kids Fiction,"$200,000,000","4,028","4,028","33,699","$110,307,189","$415,004,880","$479,175,469","$653,874,642","$1,068,879,522"


In [15]:
# Create month released column & add to dataframe. Probably don't need to do this since we can
# return the month after turning it into a datetime data type
numbers_df['Domestic Release Date'] = numbers_df['Domestic Release Date'].astype('datetime64[ns]')
numbers_df['Worldwide Release Date'] = numbers_df['Worldwide Release Date'].astype('datetime64[ns]')
month = pd.DatetimeIndex(numbers_df['Domestic Release Date']).month
numbers_df.insert(3, 'Month Released (Domestic)', month)
numbers_df.head(1)

,Domestic Release Date,Worldwide Release Date,Year Released (Domestic),Month Released (Domestic),Year Released (Worldwide),Title,Theatrical Distributor,Genre,Source,Production Method,Creative Type,Production Budget,Opening Weekend Theaters,Maximum Theaters,Theatrical Engagements,Opening Weekend Revenue,Domestic Box Office,Infl. Adj. Dom. Box Office,International Box Office,Worldwide Box Office
0,2010-06-18,2010-06-18,2010,6,2010,Toy Story 3,Walt Disney,Adventure,Original Screenplay,Digital Animation,Kids Fiction,"$200,000,000","4,028","4,028","33,699","$110,307,189","$415,004,880","$479,175,469","$653,874,642","$1,068,879,522"


In [ ]:
# Convert columns 11 + to integer
numbers_df[numbers_df.columns[11:]] = numbers_df[numbers_df.columns[11:]].apply\
(lambda x: x.str.replace('$','')).apply(lambda x: x.str.replace(',','')).astype(np.int64)

In [ ]:
# Adding available oscar count per year. Somebody please check these calculations if we use this!
numbers_df['Total Oscars Awarded in Year'] = ''
for index, row in numbers_df.iterrows():
    year = row['Year Released (Domestic)']
    if year == 1980:
        numbers_df.loc[index, 'Total Oscars Awarded in Year'] = 22
    elif year in range(1981,1995) or year == 1999:
        numbers_df.loc[index, 'Total Oscars Awarded in Year'] = 23
    elif year in range(2001,2020):
        numbers_df.loc[index, 'Total Oscars Awarded in Year'] = 25
    else:
        numbers_df.loc[index, 'Total Oscars Awarded in Year'] = 24

In [ ]:
# Title needs cleaning (remove apostrophes, colons, eplipses, "Ep. xxx:")
# The order it is replaced is sequential (ie: relacing Ep. I, followd by Ep. II returns I)

# Creating new title column so we can use original title later
numbers_df.insert(6, 'Query_Title', numbers_df['Title'])

# Replacing characters
numbers_df[numbers_df.columns[6:7]] = numbers_df[numbers_df.columns[6:7]].apply\
(lambda x: x.str.replace(":",'')).apply(lambda x: x.str.replace("Ep.","Episode"))

In [ ]:
# Limit movies to 40 per year - defined as top 40 by adjusted gross ***
numbers_df = numbers_df.sort_values(['Year Released (Domestic)', 'Infl. Adj. Dom. Box Office'],
                                    ascending = [True, False])
numbers_df = numbers_df.reset_index(drop = True)

In [ ]:
# Still limiting...
numbers_df['Year Index'] = ''
year_compare = 1980
count = 0
for index, row in numbers_df.iterrows():
    year = row['Year Released (Domestic)']
    if year == year_compare:
        count += 1
        numbers_df.loc[index, 'Year Index'] = count
    else:
        count = 1
        numbers_df.loc[index, 'Year Index'] = count
        year_compare += 1

In [ ]:
# ... a little more & voila!
top_40_df = numbers_df.loc[(numbers_df['Year Index'] <=40), ['Title', 'Query_Title',
                                                             'Domestic Release Date',
                                                             'Year Released (Domestic)',
                                                             'Month Released (Domestic)',
                                                             'Infl. Adj. Dom. Box Office',
                                                             'Domestic Box Office',
                                                             'Genre', 'Theatrical Distributor',
                                                             'Total Oscars Awarded in Year']]
top_40_df = top_40_df.sort_values('Infl. Adj. Dom. Box Office', ascending = False)
top_40_df = top_40_df.reset_index(drop = True)
top_40_df.to_csv('DataFiles/TheNumbers_Cleaned.csv')
print(top_40_df.shape)
top_40_df.head(1)

In [ ]:
# *********  This is the end of data_cleaning & start of request tests ***********

In [ ]:
# Creating dataframe to hold subset request data
omdb_df = top_40_df.copy()
omdb_df['Awards'] = ''
omdb_df['Metascore'] = ''
omdb_df['IMDB'] = ''
omdb_df['Rotten Tomatoes'] = ''
omdb_df['Rated'] = ''
omdb_df['Director'] = ''
omdb_df['Runtime'] = ''
omdb_df['Country'] = ''

In [ ]:
# *** The following cells are just for testing and can eventually be removed from code

In [ ]:
# REQUEST TESTING (Okay to remove cell)
# Sample JSON in case you want to run one specific movie title
# movie_title = "dead man's chest"
# params = {'type': 'movie', 'apikey': OMB_api_key, 't': movie_title}
# url = 'http://www.omdbapi.com/?t='
# response = requests.get(url, params).json()
# pprint(response)

In [ ]:
# REQUEST TESTING (Okay to remove cell)
# Subset dataframe for request testing
test_subset = omdb_df.iloc[25:39, :]


In [ ]:
# REQUEST TESTING (Okay to remove cell)
# Testing Requests on subset.
# If one of the values within a found movie is missing, it stops inputting data into DF after that
#     and I told it to print that so we know (uncomment the metascore row to view this)
# If we find a lot of missing movies, we could look into adding a year parameter.
#     It looks like it returns the first movie found (ie: 'Star Wars' returns 'Star Wars IV')

params = {"type": "movie", "apikey": OMB_api_key}
url = "http://www.omdbapi.com/?t="
count = 0
for index, row in test_subset.iterrows():
    params["t"] = row["Query_Title"]
    response = requests.get(url, params).json()
    if response['Response'] == 'True':
        try:
            omdb_df.loc[index, 'Awards'] = response['Awards']
            omdb_df.loc[index, 'Metascore'] = response['Metascore']
            omdb_df.loc[index, 'IMDB'] = response['imdbRating']
            omdb_df.loc[index, 'Rotten Tomatoes'] = response['Ratings'][1]['Value']
            omdb_df.loc[index, 'Rated'] = response['Rated']
            omdb_df.loc[index, 'Director'] = response['Director']
            omdb_df.loc[index, 'Runtime'] = response['Runtime']
            omdb_df.loc[index, 'Country'] = response['Country']
        except:
            print(f'{row.Query_Title.upper()} (row {count}) has missing data')
        count += 1
    else:
        print(f'{row.Query_Title.upper()} (row {count}) was not found')
        count += 1

In [ ]:
test_subset.head()

In [ ]:
# ***** API requests All Data *****

# params = {"type": "movie", "apikey": OMB_api_key}
# url = "http://www.omdbapi.com/?t="
# count = 0
# for index, row in omdb_df.iterrows():
#     params['t'] = row["Query_Title"]
#     response = requests.get(url, params).json()
#     if response['Response'] == 'True':
#         try:
#             omdb_df.loc[index, 'Awards'] = response['Awards']
#             omdb_df.loc[index, 'Metascore'] = response['Metascore']
#             omdb_df.loc[index, 'IMDB'] = response['imdbRating']
#             omdb_df.loc[index, 'Rotten Tomatoes'] = response['Ratings'][1]['Value']
#             omdb_df.loc[index, 'Rated'] = response['Rated']
#             omdb_df.loc[index, 'Director'] = response['Director']
#             omdb_df.loc[index, 'Runtime'] = response['Runtime']
#             omdb_df.loc[index, 'Country'] = response['Country']
#         except:
#             print(f'{row.Query_Title.upper()} (row {count}) has missing data')
#         count += 1
#     else:
#         print(f'{row.Query_Title.upper()} (row {count}) was not found')
#         count += 1

In [ ]:
# print(omdb_df.shape)
# omdb_df.head(1)

In [ ]:
# # Saving what we have so far
# omdb_df.to_csv('DataFiles/First_API_Run_BETA.csv', index=False)

# Loading for demo purposes
omdb_df = pd.read_csv('DataFiles/First_API_Run_BETA.csv')
omdb_df.head(50)

In [ ]:
# Nicole's code ends here

In [ ]:
# jason's code starts here

In [ ]:
# Creating 2 files one for data already populated (clean_test_df) for John to work with Analysis
# the second is for Marianne and I to parse through and find on OMDB
clean_test_df = omdb_df.dropna()
# clean_test_df.head(50)
clean_test_df.to_csv('DataFiles/clean_test_data.csv', index=False)

In [ ]:
to_be_cleaned_df = omdb_df[pd.isnull(omdb_df['Awards'])]
to_be_cleaned_df.head()
to_be_cleaned_df.shape
to_be_cleaned_df.to_csv('DataFiles/to_be_clean_data.csv', index=False)

In [ ]:
#jason's code ends here

In [ ]:
# marianne's code starts here

In [35]:
# take the last half of data to be cleaned up, based on 149 total rows of data
cleaning_df = pd.read_csv('DataFiles/to_be_clean_data.csv')
second_half_df = cleaning_df.tail(74)
second_half_df.head(20)

,Title,Query_Title,Domestic Release Date,Year Released (Domestic),Month Released (Domestic),Infl. Adj. Dom. Box Office,Domestic Box Office,Genre,Theatrical Distributor,Total Oscars Awarded in Year,Awards,Metascore,IMDB,Rotten Tomatoes,Rated,Director,Runtime,Country
75,The Divine Secrets of the Ya-Ya Siste…,The Divine Secrets of the Ya-Ya Siste…,2002-06-07,2002,6,109110734,69586544,Comedy,Warner Bros.,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,The Conjuring 2: The Enfield Poltergeist,The Conjuring 2 The Enfield Poltergeist,2016-06-10,2016,6,107919283,102470008,Horror,Warner Bros.,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,"See No Evil, Hear No Evil","See No Evil, Hear No Evil",1989-05-12,1989,5,107082447,46900000,Comedy,Sony Pictures,23,NaN,44.0,6.9,28%,R,Arthur Hiller,103 min,USA
78,Austin Powers: International Man of M…,Austin Powers International Man of M…,1997-05-02,1997,5,106946216,53883989,Comedy,New Line,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,Halloween: H2O,Halloween H2O,1998-08-05,1998,8,106914760,55041738,Horror,Miramax,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,Kill Bill: Volume 1,Kill Bill Volume 1,2003-10-10,2003,10,105883416,70098138,Action,Miramax,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,Sharky's Machine,Sharky's Machine,1981-12-18,1981,12,105585738,32984898,Thriller/Suspense,Warner Bros.,23,NaN,58.0,6.4,86%,R,Burt Reynolds,122 min,USA
82,Prince of Persia: Sands of Time,Prince of Persia Sands of Time,2010-05-28,2010,5,104793487,90759676,Action,Walt Disney,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83,Under Siege 2: Dark Territory,Under Siege 2 Dark Territory,1995-07-14,1995,7,104763078,50024083,Action,Warner Bros.,24,NaN,52.0,5.5,35%,R,Geoff Murphy,100 min,USA
84,Brewster's Millions,Brewster's Millions,1985-05-22,1985,5,104700838,40800000,Comedy,Universal,23,NaN,37.0,6.5,35%,PG,Walter Hill,102 min,USA


In [36]:
# pull out lines that were mistakenly included into the nan dataset based on awards stats
# and save for re-merging later 

m_clean_awards_df = second_half_df.dropna(subset=['IMDB'])
m_clean_awards_df.shape

(57, 18)

In [37]:
#create a new dataframe with the movies that actually need cleaning
m_cleaning_df = second_half_df[pd.isnull(second_half_df['IMDB'])]

m_cleaning_df.shape

(17, 18)

In [38]:
#overwrite cells with new query inputs
m_cleaning_df.at[75,'Query_Title'] = 'Divine Secrets'
m_cleaning_df.at[76,'Query_Title'] = 'conjuring 2'
m_cleaning_df.at[78,'Query_Title'] = 'austin powers'
m_cleaning_df.at[79,'Query_Title'] = 'Halloween H20'
m_cleaning_df.at[80,'Query_Title'] = 'Kill Bill: Vol. 1'
m_cleaning_df.at[82,'Query_Title'] = 'Prince of Persia'
m_cleaning_df.at[85,'Query_Title'] = 'Nightmare On Elm Street 3'
m_cleaning_df.at[87,'Query_Title'] = 'Percy Jackson & the Olympians'
m_cleaning_df.at[90,'Query_Title'] = 'Barnyard'
m_cleaning_df.at[91,'Query_Title'] = 'Planes'
m_cleaning_df.at[92,'Query_Title'] = 'City Slickers II'
m_cleaning_df.at[99,'Query_Title'] = "don't Breathe"
m_cleaning_df.at[100,'Query_Title'] = 'John Wick Chapter 2'
m_cleaning_df.at[104,'Query_Title'] = 'Friday the 13th: The Final Chapter'
m_cleaning_df.at[124,'Query_Title'] = "can't buy me love"
m_cleaning_df.at[125,'Query_Title'] = 'madea family funeral'
m_cleaning_df.at[146,'Query_Title'] = 'A Nightmare on Elm Street 2'

m_cleaning_df


,Title,Query_Title,Domestic Release Date,Year Released (Domestic),Month Released (Domestic),Infl. Adj. Dom. Box Office,Domestic Box Office,Genre,Theatrical Distributor,Total Oscars Awarded in Year,Awards,Metascore,IMDB,Rotten Tomatoes,Rated,Director,Runtime,Country
75,The Divine Secrets of the Ya-Ya Siste…,Divine Secrets,2002-06-07,2002,6,109110734,69586544,Comedy,Warner Bros.,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,The Conjuring 2: The Enfield Poltergeist,conjuring 2,2016-06-10,2016,6,107919283,102470008,Horror,Warner Bros.,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,Austin Powers: International Man of M…,austin powers,1997-05-02,1997,5,106946216,53883989,Comedy,New Line,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,Halloween: H2O,Halloween H20,1998-08-05,1998,8,106914760,55041738,Horror,Miramax,24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,Kill Bill: Volume 1,Kill Bill: Vol. 1,2003-10-10,2003,10,105883416,70098138,Action,Miramax,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,Prince of Persia: Sands of Time,Prince of Persia,2010-05-28,2010,5,104793487,90759676,Action,Walt Disney,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,A Nightmare On Elm Street 3: Dream Wa…,Nightmare On Elm Street 3,1987-02-27,1987,2,104364770,44793222,Horror,New Line,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,Percy Jackson & the Olympians: The Li…,Percy Jackson & the Olympians,2010-02-12,2010,2,102494196,88768303,Adventure,20th Century Fox,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,Barnyard: The Original Party Animals,Barnyard,2006-08-04,2006,8,101223916,72779000,Adventure,Paramount Pictures,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
91,Disney Planes,Planes,2013-08-09,2013,8,101165347,90282580,Adventure,Walt Disney,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
#re-run the API for the updated query names


params = {"type": "movie","apikey": OMB_api_key}
url = "http://www.omdbapi.com/?t="
count = 0
for index, row in m_cleaning_df.iterrows():
    params['t'] = row["Query_Title"]
    params['y'] = row["Year Released (Domestic)"]
    response = requests.get(url, params).json()
    if response['Response'] == 'True':
        try:
            m_cleaning_df.loc[index, 'Awards'] = response['Awards']
            m_cleaning_df.loc[index, 'Metascore'] = response['Metascore']
            m_cleaning_df.loc[index, 'IMDB'] = response['imdbRating']
            m_cleaning_df.loc[index, 'Rotten Tomatoes'] = response['Ratings'][1]['Value']
            m_cleaning_df.loc[index, 'Rated'] = response['Rated']
            m_cleaning_df.loc[index, 'Director'] = response['Director']
            m_cleaning_df.loc[index, 'Runtime'] = response['Runtime']
            m_cleaning_df.loc[index, 'Country'] = response['Country']
        except:
            print(f'{row.Query_Title.upper()} (row {count}) has missing data')
        count += 1
    else:
        print(f'{row.Query_Title.upper()} (row {count}) was not found')
        count += 1

C:\Users\mmidw\anaconda3\envs\NewPythonData\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [42]:
#concatenate both of marianne's dataframes back together

frames = [m_clean_awards_df, m_cleaning_df]
m_complete_df = pd.concat(frames)
m_complete_df.head(50)

#save the completed data to a csv for later concatenation

,Title,Query_Title,Domestic Release Date,Year Released (Domestic),Month Released (Domestic),Infl. Adj. Dom. Box Office,Domestic Box Office,Genre,Theatrical Distributor,Total Oscars Awarded in Year,Awards,Metascore,IMDB,Rotten Tomatoes,Rated,Director,Runtime,Country
77,"See No Evil, Hear No Evil","See No Evil, Hear No Evil",1989-05-12,1989,5,107082447,46900000,Comedy,Sony Pictures,23,NaN,44,6.9,28%,R,Arthur Hiller,103 min,USA
81,Sharky's Machine,Sharky's Machine,1981-12-18,1981,12,105585738,32984898,Thriller/Suspense,Warner Bros.,23,NaN,58,6.4,86%,R,Burt Reynolds,122 min,USA
83,Under Siege 2: Dark Territory,Under Siege 2 Dark Territory,1995-07-14,1995,7,104763078,50024083,Action,Warner Bros.,24,NaN,52,5.5,35%,R,Geoff Murphy,100 min,USA
84,Brewster's Millions,Brewster's Millions,1985-05-22,1985,5,104700838,40800000,Comedy,Universal,23,NaN,37,6.5,35%,PG,Walter Hill,102 min,USA
86,Bachelor Party,Bachelor Party,1984-06-29,1984,6,104211741,38435947,Comedy,20th Century Fox,23,NaN,56,6.3,54%,R,Neal Israel,105 min,USA
88,Hard To Kill,Hard To Kill,1990-02-09,1990,2,102285404,47381386,Action,Warner Bros.,23,NaN,41,5.8,38%,R,Bruce Malmuth,96 min,USA
89,D2: The Mighty Ducks,D2 The Mighty Ducks,1994-03-25,1994,3,101827034,45604206,Comedy,Walt Disney,23,NaN,NaN,6.1,20%,PG,Sam Weisman,106 min,USA
93,Passenger 57,Passenger 57,1992-11-06,1992,11,96732066,44065653,Action,Warner Bros.,23,NaN,NaN,5.9,24%,R,Kevin Hooks,84 min,USA
94,Biloxi Blues,Biloxi Blues,1988-03-25,1988,3,95721048,43184798,Comedy,Universal,23,NaN,61,6.6,78%,PG-13,Mike Nichols,106 min,USA
95,"Fort Apache, The Bronx","Fort Apache, The Bronx",1981-01-01,1981,1,95687769,29200000,NaN,20th Century Fox,23,NaN,49,6.7,86%,R,Daniel Petrie,125 min,USA


In [ ]:
# marianne's code ends here

In [ ]:
# NEXT STEPS
# Pull out NaN values from omdb_df and resave
# Create new dataframes with only NaN values & figure out how to make successful API calls on them
    # (might be a series of datframes & API calls after tweaking key words or maybe adding variable for year)
# Pull out oscar nominations and wins
# Save & review final dataframe